In [12]:
import keras 
import pandas as pd
import matplotlib.pyplot as plt 
import keras 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image 
import os
import numpy as np
import h5py
import math
import dlib
import cv2
import easydict
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder 

In [3]:
args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}

In [4]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor(args["shape_predictor"])

In [5]:
def rect_to_bb(rect):
    x=rect.left()
    y=rect.top()
    w=rect.right()-x
    h=rect.bottom()-y
    return (x,y,w,h)

In [6]:
def shape_to_np(shape,dtype='int'):
    coords=np.zeros((68,2),dtype=dtype)
    for i in range(0,68):
        coords[i]=(shape.part(i).x,shape.part(i).y)
    return coords

In [7]:
def preprocess_dis(dis_array):
    x=dis_array-np.mean(dis_array)
    if np.std(dis_array)!=0:
        x/=np.std(dis_array)
    return x

In [8]:
def convert_points_to_distance(points):

    mid_x=(points[14,0]+points[18,0])/2
    mid_y=(points[14,1]+points[18,1])/2
    mid_point=np.array([mid_x,mid_y])
    points=(points-mid_point)**2
    distances_array=np.sqrt((points[:,0]+points[:,1]))
    
    return preprocess_dis(distances_array)
    

In [9]:
def face_landmarks(image):
    #all_images=[]
    #for image in images:
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects=detector(gray,1)
    shape=np.zeros((20,2))
    for rect in rects:
        shape=predictor(gray,rect)
        shape=shape_to_np(shape)
        shape=shape[48:68]
    #all_images.append(shape)

    return shape   

In [10]:
def empty_distance(number):
    shape=np.zeros((20,number))
    return shape

In [11]:
lip_data = []

In [ ]:
labels=[]
datagen=ImageDataGenerator()
train_dir="C:\\Users\\this pc\\Desktop\\Minor\\Data"
for label in os.listdir(train_dir):
    labels_dir=os.path.join(train_dir,label)
    for repeat in os.listdir(labels_dir):       
        word_dir=os.path.join(labels_dir,repeat)
        for sub in os.listdir(word_dir):
            first_dir = os.path.join(word_dir, sub)
            print(repeat)
            if(repeat == "phrases"):
                val = "P"
            else:
                val = "W"
            for sec_sub in os.listdir(first_dir):
                sec_dir = os.path.join(first_dir, sec_sub)
                print("currently in %", sec_dir)
                images_array=[]
                num_images=0
                for image_dir in os.listdir(sec_dir):
                    if num_images==10:
                        break
                    image=cv2.imread(os.path.join(sec_dir,image_dir))
                    points=face_landmarks(image)
                    points=convert_points_to_distance(points)
                    #if points is not None:
                    images_array.append(points)
                    print(points)
                    num_images=num_images+1
                if num_images<10:
                    np.append(images_array,empty_distance(10-num_images))
                lip_data=np.append(lip_data,np.array(images_array))
                label1 = str(val) + str(int(sub)-1)
                labels.append(label1)
lip_data=lip_data.reshape(400,10,20)

In [ ]:
data_order = 'tf'

In [ ]:
data_shape = (len(lip_data), 10, 20)

In [ ]:
hdf5_file = h5py.File(hdf5_path, mode='w')